# 问题 G: You Shall not AK

时间限制: 8 Sec  内存限制: 256 MB

[https://acm.sustech.edu.cn/onlinejudge/problem.php?cid=1084&pid=6](https://acm.sustech.edu.cn/onlinejudge/problem.php?cid=1084&pid=6)

## 题目描述

Given $n$ strings containing only lowercase letters, and the $i_{th}$  string is $s_i$.

$f(s,t)$ represents the maximum $i$ satisfy $s[:i]=t[-i:]$, and $f(s,t)=0$ if such $i$ doesn't exist. 

Please calculate:
$$
\sum_{i=1}^{n} \sum_{j=1}^{n}(i \times j \times f(s_i,s_j)) (\mod 998244353)
$$

## 输入

The first line is the integer $n$. $(1 \le n \le 100000)$

The following n lines contains one string $s_i$. $(1 \le |si|,1 \le i \le n,\sum_{i=1}^{n}|si| \le 1000000)$

## 输出

One line an integer, representing the answer.

## 样例输入
```
2
ab
aba
```
## 样例输出
```
20
```

# 简要题解

## Trie Tree + Suffix Tree, $O(N)$.

先对字符串列表,翻转建Trie Tree, 同时要标记一下每条路径的weight,也可以理解成流量.

假设给定一个字符串s, 我们可以求出以s结尾的字符串的权重和是多少.

这道题目更复杂一点, 给定一个字符串s, 我们要求$s[:i] = t[-i:]$. 但依然可以套刚刚这个简化版本来解决.

方法就是对s翻转建一个suffix tree, 那么suffix tree上的每一条路径都对应着一个substring, $s[i:-1:-1]$.

DFS遍历suffix tree的时候, 同时在trie tree上查询当前path的权重和.

当然还有些小细节要处理,比如suffix tree上有相同prefix的substring,需要做些加加减减来避免重复计算.

P.S. suffix tree的空间是$O(N)$, 建树时间也可以达到$O(N)$. 

这里我是暴力建的suffix tree, 并没有优化到$O(N)$, 反正思路就这样. 

In [2]:
def split(word): 
    return [ord(char) - ord('a') for char in word]  

class Node:
    def __init__(self):
        self.c = None
        self.w = 0
        self.done = False
    def add(self, s, w):
        self.w += w
        if s == []:
            self.done = True
            return
        child = self.get(s[0])
        child.add(s[1:],w)
    def get(self, x):
        if self.c == None:
            self.c = [None for _ in range(26)]
        if self.c[x] == None:
            self.c[x] = Node()
        return self.c[x]
    def print(self, prefix):
        print(prefix + " " + str(self.w) + " " + str(self.done))
        if self.c != None:
            for i in range(26):
                if self.c[i] == None:
                    continue
                self.c[i].print(prefix + chr(97 + i))

def build_tree(strs):
    root = Node()
    for i, s in enumerate(strs):
        root.add(split(s), i + 1)
    return root

def get_suffix(s):
    ret = []
    for i in range(len(s)):
        ret.append(s[i:])
    return ret
        
ss = ["abc", "bc", "aab"]
ss_rev = [s[::-1] for s in ss]
root = build_tree(ss_rev)
root.print("$")


s = "abc"
suffix = get_suffix(s[::-1])
print(suffix)
s_root = build_tree(suffix)
s_root.print("$")

def calc(suffix, trie, depth):
    w = 0
    ans = 0
    flag = True
    
    if suffix.done == True:
        flag = False
        w = trie.w
        ans = trie.w * depth
        
    if suffix.c != None and trie.c != None:
        for i in range(26):
            if suffix.c[i] == None or trie.c[i] == None:
                continue
            cw, cans = calc(suffix.c[i], trie.c[i], depth + 1)
            if flag:
                w += cw
                ans += cans
            else:
                ans += cans
                ans -= cw * depth
    # print(depth, w, ans)
    return w, ans
    
calc(s_root, root, 0)

def solve(ss, s):
    ss_rev = [s[::-1] for s in ss]
    root = build_tree(ss_rev)
    #root.print("$")
    suffix = get_suffix(s[::-1])
    s_root = build_tree(suffix)
    #s_root.print("#")
    w, ans = calc(s_root, root, 0)
    return ans

$ 6 False
$b 3 False
$ba 3 False
$baa 3 True
$c 3 False
$cb 3 True
$cba 1 True
['cba', 'ba', 'a']
$ 6 False
$a 3 True
$b 2 False
$ba 2 True
$c 1 False
$cb 1 False
$cba 1 True


In [16]:
ss = ["abab", "aba", "ababcab", "abaa", "abcab"]
s = "abab"

def solve_bf(ss, s):
    ret = 0
    for i, t in enumerate(ss):
        for j in range(len(t)):
            if s.startswith(t[j:]):
                ret += (len(t) - j) * (i + 1)
                break
    return ret

print(solve(ss, s), solve_bf(ss, s))

def solve_all(ss):
    ss_rev = [s[::-1] for s in ss]
    root = build_tree(ss_rev)
    ret = 0
    for i, s in enumerate(ss):
        suffix = get_suffix(s[::-1])
        s_root = build_tree(suffix)
        w, ans = calc(s_root, root, 0)
        ret += ans * (i+1)
    return ret

def solve_all_bf(ss):
    ret = 0
    for i, s in enumerate(ss):
        ret += solve_bf(ss, s) * (i+1)
    return ret

print(solve_all(ss), solve_all_bf(ss))

print(solve_all(["ab", "aba"]), solve_all_bf(["ab", "aba"]))

"abc"[2::-1]

30 30
621 621
20 20


'cba'